In [ ]:
import sys
sys.path.append('../')

import pipeline.sql as plsql

import pandas as pd
import numpy as np

from plotnine import *
import pandas as pd
from mizani.breaks import date_breaks
from mizani.formatters import date_format
from datetime import date

theme_set(theme_bw())

engine = plsql.create_engine('../config.yaml')
role = 'set role direccion_trabajo_inspections_write'

In [ ]:
qry = """{};
    select  count( distinct entity_id) 
    from cleaned.inspections_se;
""".format(role)
#plsql.query(qry, engine)

In [ ]:
# Copper
qry = """{};
    SELECT avg_copper_price,
    DATE_TRUNC('month', month_plus_one) as month_year
    FROM cleaned.copper_formatted
    order by month_plus_one;
    """.format(role)
tab_copper = plsql.query(qry, engine)
print(tab_copper.shape)
tab_copper.head()

In [ ]:
(ggplot(tab_copper, aes('month_year', 'avg_copper_price')) + 
    geom_line(color = 'blue', size = 1) + 
    ylab('Copper Price') + 
    xlab('Date') + 
    ylim(0, 6))

In [ ]:
# Macro econ
qry = """{};
    SELECT *
    FROM cleaned.macro_month
    limit 3;
    """.format(role)
plsql.query(qry, engine)


In [ ]:
print(tab_copper.shape)
tab_copper.head()

In [ ]:
# Inspectors
qry = """{};
    SELECT date_monthyear as month_year, 
    count(distinct funasignadomask) as inspectors
    FROM cleaned.inspections_se
    where tipotermino != 'empleador no ubicado'
    group by month_year;
    """.format(role)
tab_inspt = plsql.query(qry, engine)
tab_inspt.month_year = pd.to_datetime(tab_inspt.month_year)
print(tab_inspt.shape)
tab_inspt.head()

In [ ]:
tab_inspt.head()

## Total month year

In [ ]:
qry = """{};
    SELECT date_monthyear as month_year, 
        count(distinct entity_id) as infracted_entities,
        count(entity_id) as infractions_total
    FROM cleaned.inspections_se
    WHERE tipotermino != 'empleador no ubicado'
        and infractor = 'true'
    GROUP BY month_year;
    """.format(role)
tab_inf_tot = plsql.query(qry, engine)

In [ ]:
qry = """{};
    SELECT date_monthyear as month_year, 
    count(distinct entity_id) as inspected_entities,
    count(entity_id) as inspections_total,
    count(distinct funasignadomask) as inspectors
    FROM cleaned.inspections_se
    where tipotermino != 'empleador no ubicado'
    group by month_year;""".format(role)
tab_ins_tot = plsql.query(qry, engine)

In [ ]:
tab_inf_tot.month_year = pd.to_datetime(tab_inf_tot.month_year)
tab_ins_tot.month_year = pd.to_datetime(tab_ins_tot.month_year)
tab_union_tot = (tab_ins_tot.merge(tab_inf_tot, how = 'inner', 
                              left_on = ['month_year'], 
                              right_on = ['month_year']))
tab_union_tot.head()

In [ ]:
tab_union_tot['prop_infra'] = 100*tab_union_tot['infracted_entities']/tab_union_tot['inspected_entities']
tab_union_tot['prop_inspc'] = 100*tab_union_tot['inspected_entities']/tab_union_tot['inspectors']

In [ ]:
tab_union_tot['prop_infra'].describe()

In [ ]:
(ggplot(tab_union_tot, aes('month_year', 'inspected_entities')) + 
    geom_line(size =1) + 
    ylab('Inspected entities) + 
    xlab('Date') +  
    ylim(0, 60))

In [ ]:
(ggplot(tab_union_tot, aes('month_year', 'prop_infra')) + 
    geom_line(size =1) + 
    ylab('Infracted/Inspected') + 
    xlab('Date') +  
    ylim(0, 60))

In [ ]:
(ggplot(tab_union_tot, aes('month_year', 'prop_inspc')) + 
    geom_line(size =1) + 
    ylab('Entities by Infractor') + 
    xlab('Date'))

In [ ]:
tab_union_tot.prop_inspc.describe()

## Total by program

In [ ]:
qry = """{};
    SELECT 
        case when solicitante = 'Por Programa' then 'proactive' else 'reactive' end as program,
        count(distinct entity_id) as infracted_entities,
        count(entity_id) as infractions_total
    FROM cleaned.inspections_se
    WHERE tipotermino != 'empleador no ubicado'
        and infractor = 'true'
    GROUP BY program;
    """.format(role)
tab_inf_cprog = plsql.query(qry, engine)

In [ ]:
qry = """{};
    SELECT 
        case when solicitante = 'Por Programa' then 'proactive' else 'reactive' end as program,
        count(distinct entity_id) as inspected_entities,
        count(entity_id) as inspections_total,
        count(distinct funasignadomask) as inspectors
    FROM cleaned.inspections_se
    where tipotermino != 'empleador no ubicado'
    GROUP BY program;
    """.format(role)
tab_ins_cprog = plsql.query(qry, engine)

In [ ]:
tab_inf_cprog

In [ ]:
tab_ins_cprog

## Total

In [ ]:
qry = """{};
    SELECT 
        count(distinct entity_id) as infracted_entities,
        count(entity_id) as infractions_total
    FROM cleaned.inspections_se
    WHERE tipotermino != 'empleador no ubicado'
        and infractor = 'true';
    """.format(role)
tab_inf_com = plsql.query(qry, engine)

In [ ]:
qry = """{};
    SELECT 
        count(distinct entity_id) as inspected_entities,
        count(entity_id) as inspections_total,
        count(distinct funasignadomask) as inspectors
    FROM cleaned.inspections_se
    where tipotermino != 'empleador no ubicado';
    """.format(role)
tab_ins_com = plsql.query(qry, engine)

In [ ]:
tab_inf_com

In [ ]:
tab_ins_com

## Total  year

In [ ]:
qry = """{};
    SELECT 
        DATE_TRUNC('year', date_monthyear) as year, 
        count(distinct entity_id) as infracted_entities,
        count(entity_id) as infractions_total
    FROM cleaned.inspections_se
    WHERE tipotermino != 'empleador no ubicado'
        and infractor = 'true'
    GROUP BY year;
    """.format(role)
tab_inf_yr = plsql.query(qry, engine)

In [ ]:
qry = """{};
    SELECT 
        DATE_TRUNC('year', date_monthyear) as year, 
        count(distinct entity_id) as inspected_entities,
        count(entity_id) as inspections_total,
        count(distinct funasignadomask) as inspectors
    FROM cleaned.inspections_se
    WHERE tipotermino != 'empleador no ubicado'
    GROUP BY year;""".format(role)
tab_ins_yr = plsql.query(qry, engine)

In [ ]:
tab_inf_yr.head()

In [ ]:
tab_ins_yr.head()

In [ ]:
tab_inf_yr.year = pd.to_datetime(tab_inf_yr.year)
tab_ins_yr.year = pd.to_datetime(tab_ins_yr.year)
tab_union_yr = tab_ins_yr.merge(tab_inf_yr, how = 'inner', left_on = ['year'], right_on = ['year'])

In [ ]:
tab_union_yr['prop'] = 100*tab_union_yr['infracted_entities']/tab_union_yr['inspected_entities']
tab_union_yr['infract'] = 100*tab_union_yr['infractions_total']/tab_union_yr['inspections_total']
tab_union_yr['insp_insp'] = tab_union_yr['inspections_total']/tab_union_yr['inspectors']

In [ ]:
tab_union_yr

In [ ]:
tab_union_yr.inspected_entities.mean()
tab_union_yr.inspections_total.mean()
tab_union_yr.infracted_entities.mean()
tab_union_yr.infractions_total.mean()
tab_union_yr.inspectors.mean()
tab_union_yr.infract.mean()
tab_union_yr.insp_insp.mean()

## Program by year

In [ ]:
qry = """{};
    SELECT 
        DATE_TRUNC('year', date_monthyear) as year, 
        case when solicitante = 'Por Programa' then 'proactive' else 'reactive' end as program,
        count(distinct entity_id) as infracted_entities,
        count(entity_id) as infractions_total
    FROM cleaned.inspections_se
    WHERE tipotermino != 'empleador no ubicado'
        and infractor = 'true'
    GROUP BY year, program;
    """.format(role)
tab_inf_progyr = plsql.query(qry, engine)

In [ ]:
qry = """{};
    SELECT 
        DATE_TRUNC('year', date_monthyear) as year, 
        case when solicitante = 'Por Programa' then 'proactive' else 'reactive' end as program,
        count(distinct entity_id) as inspected_entities,
        count(entity_id) as inspections_total,
        count(distinct funasignadomask) as inspectors
    FROM cleaned.inspections_se
    WHERE tipotermino != 'empleador no ubicado'
    GROUP BY year, program;""".format(role)
tab_ins_progyr = plsql.query(qry, engine)

In [ ]:
tab_inf_progyr.head()

In [ ]:
tab_ins_progyr.head()

In [ ]:
tab_inf_progyr.year = pd.to_datetime(tab_inf_progyr.year)
tab_ins_progyr.year = pd.to_datetime(tab_ins_progyr.year)
tab_union_progyr = (tab_ins_progyr.merge(tab_inf_progyr, how = 'inner',
                                        left_on = ['year', 'program'], 
                                        right_on = ['year', 'program']).
                    reindex())

In [ ]:
tab_union_progyr['prop'] = 100*tab_union_progyr['infracted_entities']/tab_union_progyr['inspected_entities']
tab_union_progyr['infract'] = 100*tab_union_progyr['infractions_total']/tab_union_progyr['inspections_total']
tab_union_progyr['inspinsp'] = tab_union_progyr['inspections_total']/tab_union_progyr['inspectors']

In [ ]:
tab_union_progyr[tab_union_progyr.program == 'proactive'].sort_values(['program', 'year']).head()

In [ ]:
tab_melt_yr = pd.melt(tab_union_progyr, id_vars=['year', 'program'], 
        value_vars=['inspected_entities', 'inspections_total',
                    'inspectors', 'infracted_entities', 'infractions_total', 'prop', 
                    'infract', 'insp_insp', 'insp_day'])
tab_melt_yr.groupby(['program', 'variable']).mean()

In [ ]:
tab_union_progyr[tab_union_progyr.year == '2015-01-01 '].sort_values(['program', 'year'])

## Program by month and year

In [ ]:
qry = """{};
    SELECT date_monthyear as month_year, 
        case when solicitante = 'Por Programa' then 'proactive' else 'reactive' end as program,
        count(distinct entity_id) as infracted_entities,
        count(entity_id) as infractions_total
    FROM cleaned.inspections_se
    WHERE tipotermino != 'empleador no ubicado'
        and infractor = 'true'
    GROUP BY month_year, program;
    """.format(role)
tab_inf = plsql.query(qry, engine)

In [ ]:
qry = """{};
    SELECT date_monthyear as month_year, 
    count(distinct entity_id) as inspected_entities,
    count(entity_id) as inspections_total,
    count(distinct funasignadomask) as inspectors,
    case when solicitante = 'Por Programa' then 'proactive' else 'reactive' end as program
    FROM cleaned.inspections_se
    where tipotermino != 'empleador no ubicado'
    group by month_year, program;""".format(role)
tab_ins = plsql.query(qry, engine)

In [ ]:
tab_inf.head()

In [ ]:
tab_ins.head()

In [ ]:
print(tab_inf.shape)
tab_inf.dtypes

In [ ]:
print(tab_ins.shape)
tab_ins.dtypes

In [ ]:
tab_inf.month_year = pd.to_datetime(tab_inf.month_year)
tab_ins.month_year = pd.to_datetime(tab_ins.month_year)
tab_union = tab_ins.merge(tab_inf, how = 'inner', left_on = ['month_year', 'program'], right_on = ['month_year', 'program'])

In [ ]:
tab_union['prop'] = 100*tab_union['infracted_entities']/tab_union['inspected_entities']
tab_union['infract'] = 100*tab_union['infractions_total']/tab_union['inspections_total']
tab_union['insp_insp'] = tab_union['inspections_total']/tab_union['inspectors']
tab_union['insp_day'] = tab_union['inspections_total']/20

In [ ]:
tab_union.shape
tab_union.head()

In [ ]:
tab_melt_u = pd.melt(tab_union, id_vars=['month_year', 'program'], 
        value_vars=['inspected_entities', 'inspections_total',
                    'inspectors', 'infracted_entities', 'infractions_total', 'prop', 
                    'infract', 'insp_insp', 'insp_day'])
tab_melt_u.groupby(['program', 'variable']).mean()

In [ ]:
(ggplot(tab_union, aes('month_year', 'infract', color = 'program')) + 
    geom_line(size =1) + 
    ylab('Infracted/Inspected') + 
    xlab('Date') +  
    ylim(0, 60))

# Month year total

In [ ]:
qry = """{};
    SELECT date_monthyear as month_year, 
        count(distinct entity_id) as infracted_entities,
        count(entity_id) as infractions_total
    FROM cleaned.inspections_se
    WHERE tipotermino != 'empleador no ubicado'
        and infractor = 'true'
    GROUP BY month_year;
    """.format(role)
tab_inf = plsql.query(qry, engine)

In [ ]:
qry = """{};
    SELECT date_monthyear as month_year, 
    count(distinct entity_id) as inspected_entities,
    count(entity_id) as inspections_total,
    count(distinct funasignadomask) as inspectors
    FROM cleaned.inspections_se
    where tipotermino != 'empleador no ubicado'
    group by month_year;""".format(role)
tab_ins = plsql.query(qry, engine)

In [ ]:
tab_inf.month_year = pd.to_datetime(tab_inf.month_year)
tab_ins.month_year = pd.to_datetime(tab_ins.month_year)
tab_union = tab_ins.merge(tab_inf, how = 'inner', 
                          left_on = ['month_year'],
                          right_on = ['month_year'])



In [ ]:
tab_union['prop'] = 100*tab_union['infracted_entities']/tab_union['inspected_entities']
tab_union['infract'] = 100*tab_union['infractions_total']/tab_union['inspections_total']
tab_union['insp_insp'] = tab_union['inspections_total']/tab_union['inspectors']
tab_union['insp_day'] = tab_union['inspections_total']/20

In [ ]:
tab_melt_u = pd.melt(tab_union, id_vars=['month_year'], 
        value_vars=['inspected_entities', 'inspections_total',
                    'inspectors', 'infracted_entities', 'infractions_total', 'prop', 
                    'infract', 'insp_insp', 'insp_day'])
tab_melt_u.groupby(['variable']).mean()